# First steps into Bayesian optimization
*Ivo Couckuyt*, *Joachim van der Herten*

## Introduction

Bayesian optimization is particularly useful for expensive optimization problems. This includes optimization problems where the objective (and constraints) are time-consuming to evaluate: measurements, engineering simulations, cross-hyperparameter optimization of deep learning models, etc. Another area where Bayesian optimization may provide a benefit is in the presence of (a lot of) noise.

If your problem does not satisfy these requirements other optimization algorithms might be better suited.

To setup a basic Bayesian optimization you have to:

- define your objective and specify the optimization domain
- setup a GPflow model and choose an acquisition function
- create a BayesianOptimizer

## Objective function

In [1]:
import numpy as np
from GPflowOpt.domain import ContinuousParameter


def fx(X):
    X = np.atleast_2d(X)
    # Return objective & gradient
    return np.sum(np.square(X), axis=1)[:, None]

domain = ContinuousParameter('x1', -2, 2) + ContinuousParameter('x2', -1, 2)
domain

Name,Type,Values
x1,Continuous,[-2. 2.]
x2,Continuous,[-1. 2.]


## Bayesian optimizer

In [2]:
import GPflow
from GPflowOpt.bo import BayesianOptimizer
from GPflowOpt.design import LatinHyperCube
from GPflowOpt.acquisition import ExpectedImprovement
from GPflowOpt.optim import SciPyOptimizer

# Use standard Gaussian process Regression
lhd = LatinHyperCube(21, domain)
X = lhd.generate()
Y = fx(X)
model = GPflow.gpr.GPR(X, Y, GPflow.kernels.Matern52(2, ARD=True))

# Now create the Bayesian Optimizer
alpha = ExpectedImprovement(model)
optimizer = BayesianOptimizer(domain, alpha)

# Run the Bayesian optimization
with optimizer.silent():
    r = optimizer.optimize(fx, n_iter=15)
print(r)

     fun: array([  1.69219493e-05])
 message: 'OK'
    nfev: 15
 success: True
       x: array([[ -9.60370839e-05,   4.11250850e-03]])


That's all! Your objective function has now been optimized over 15 iterations.